## Deliverable 2. Create a Customer Travel Destinations Map.

In [7]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import sys
# Import API key
import sys  
sys.path.insert(0, '.../config')
from config import g_key
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [10]:
#Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Atuona,PF,-9.8000,-139.0333,77.22,76,20,16.06,few clouds
1,1,Barrow,US,71.2906,-156.7887,35.62,93,100,13.80,moderate rain
2,2,Punta Arenas,CL,-53.1500,-70.9167,30.31,93,0,10.36,clear sky
3,3,Saint-Philippe,RE,-21.3585,55.7679,65.79,85,56,11.21,light rain
4,4,Bethel,US,41.3712,-73.4140,75.76,66,100,4.00,overcast clouds


In [12]:
#Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 70


In [14]:
#Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
#Drop any empty rows and check if you still have any
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
clean_df.count()


City_ID                142
City                   142
Country                142
Lat                    142
Lng                    142
Max Temp               142
Humidity               142
Cloudiness             142
Wind Speed             142
Current Description    142
dtype: int64

In [15]:
#Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

#Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Saint-Philippe,RE,65.79,light rain,-21.3585,55.7679,
14,Kolosovka,RU,65.03,overcast clouds,56.4678,73.6110,
19,Port Alfred,ZA,63.28,scattered clouds,-33.5906,26.8910,
34,Cap Malheureux,MU,68.47,few clouds,-19.9842,57.6142,
41,Santa Maria,BR,61.90,mist,-29.6842,-53.8069,
55,Eura,FI,60.58,clear sky,61.1333,22.1333,
59,Sibiti,CG,64.27,few clouds,-3.6819,13.3499,
64,Hovd,MN,62.33,clear sky,48.0056,91.6419,
71,Miraflores,MX,68.65,overcast clouds,19.2167,-98.8083,
78,Airai,TL,66.96,broken clouds,-8.9266,125.4092,


In [17]:
#Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name, add except for error
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [24]:
#Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].dropna()


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Saint-Philippe,RE,65.79,light rain,-21.3585,55.7679,Le four à pain
14,Kolosovka,RU,65.03,overcast clouds,56.4678,73.6110,Kolosovskaya Gostinitsa
19,Port Alfred,ZA,63.28,scattered clouds,-33.5906,26.8910,The Halyards Hotel
34,Cap Malheureux,MU,68.47,few clouds,-19.9842,57.6142,Pereybere Hotel & Spa
41,Santa Maria,BR,61.90,mist,-29.6842,-53.8069,Hotel Morotin


In [26]:
# Create the output File (CSV)
hotel_output = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(hotel_output, index_label="City_ID")

In [33]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>County</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} F</dd>
</dl>
"""
# Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

#Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [34]:
# Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display the figure 
fig



Figure(layout=FigureLayout(height='420px'))